In [1]:
#Notebook for development of the game
# Imports main modules - Player, MapGenerator, etc.
# Implements pygame controls and GUI

In [1]:
import sys, random, os, math, time
import numpy as np

#where non-dev code is located
sys.path.append('../app')

In [2]:
#install pygame
#!{sys.executable} -m pip install pygame

In [3]:
import pygame

#config stores constants and default settings
from config import Config
from map_generator import MapGenerator
from the_map import TheMap
from player import HumanPlayer, AgentPlayer
from capture_the_flag import CaptureTheFlag

#helpful pygame wrapper
import pygame_utils as pyg

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
#MAKE SURE any changes here are reflected in app/the_map.py

class TheMap():
    def __init__(self, config, new_map=False):
        self.config = config
        self.border_size = self.config.map_border_size
        self.tile_size = self.config.terrain_tile_size
        
        #make the map and get the terrain speeds associated with the map
        if new_map:
            if self.config.verbose:
                print('Creating new map')
            tile_cols, tile_rows = 40, 30
            pixel_dims = (self.tile_size * tile_cols, self.tile_size * tile_rows)
            self.tile_speeds, self.map_path = MapGenerator(self.config, seed=13).generate_map(pixel_dims, 
                                                                        save_path = self.config.maps_path)
        else:
            if self.config.verbose:
                print('Using default map')
            self.map_path = self.config.map_default_path
            self.tile_speeds = np.load(self.config.map_default_speed_array)
            
        self.middle_tile = self.tile_speeds.shape[1]//2
            
        self.blue_flag_tile = (0,0)
        self.blue_flag_area_tiles = []
        self.red_flag_tile = (0,0)
        self.red_flag_area_tiles = []
        
        self.player_tile = (0,0)
        self.blue_agent_tiles = []
        self.red_agent_tiles = []
        
        
    def set_flag_location(self, team, flag_x, flag_y):
        flag_c, flag_r = self.xy_to_cr(flag_x, flag_y)
        
        if team=='blue':
            self.blue_flag_tile = (flag_r, flag_c)
        else:
            self.red_flag_tile = (flag_r, flag_c)
        
        #get tiles in flag area
        flag_area_border_tiles = ((self.config.flag_area_size//self.tile_size) // 2) - 1
        for r in range(flag_r - flag_area_border_tiles, flag_r + flag_area_border_tiles + 1):
            for c in range(flag_c - flag_area_border_tiles, flag_c + flag_area_border_tiles + 1):
                if team=='blue':
                    self.blue_flag_area_tiles.append((r, c))
                else:
                    self.red_flag_area_tiles.append((r, c))
                    
        if self.config.verbose:
            print('%s flag is on %s, %s, in area %s' % (team, flag_r, flag_c, 
                    str(self.blue_flag_area_tiles) if team=='blue' else str(self.red_flag_area_tiles)))
                    
        
    def get_not_allowed_tiles(self):
        idx = np.where(self.tile_speeds==0)
        
        return list(zip(idx[0].tolist(), idx[1].tolist()))
    
    
    def xy_to_cr(self, x, y):
        return x // self.config.terrain_tile_size, y // self.config.terrain_tile_size
        
        
    def get_speed(self, tile_col, tile_row):
        return int(self.tile_speeds[tile_row, tile_col])
    
    
    def in_enemy_territory(self, team, tile_col):
        if team=='blue':
            return tile_col > self.middle_tile
        else:
            return tile_col < self.middle_tile
        
        
    def in_flag_area(self, team, tile_col, tile_row):
        if team=='blue':
            return (tile_row, tile_col) in self.blue_flag_area_tiles
        else:
            return (tile_row, tile_col) in self.red_flag_area_tiles
        

In [ ]:
#MAKE SURE any changes here are reflected in app/player.py

class Player():
    def __init__(self, x, y, idx, team, the_map, config):
        '''
        x,y - the player/agent sprite starting location on the map
        team - blue or red
        the_map - a reference to the global map with speeds, flag locations, player locations
        config - configurable settings
        '''
        self.config = config
        
        #w,a,d,s are the keyboard directions for up, left, right, down
        self.actions = ['w', 'a', 'd', 's']
        
        #for debugging
        self.speed_terr = {v:k for k,v in self.config.terrain_speeds.items()}
        
        #configured values
        
        #TODO - implement variable default speeds so some players are naturally faster
        #self.max_speed = self.config.player_max_speed
        
        #TODO implement energy countdown to promote teamwork through revivals
        #self.max_energy = self.config.player_max_energy
        #self.min_energy = self.config.player_min_energy
        #self.energy = self.config.player_max_energy
        
        self.player_idx = idx
        self.team = team
        self.the_map = the_map
        self.sprite = None
        
        #current status
        self.speed = self.config.player_max_speed
        self.has_flag = False
        self.is_incapacitated = False
        self.incapacitated_countdown = 0
        self.in_enemy_territory = False
        self.in_flag_area = False
        
        self.x = x
        self.y = y
        self.tile_col, self.tile_row = self.the_map.xy_to_cr(x, y)
        
    
    def update(self, frame):
        #get movement action based on map, players, flag percepts
        action = self.get_action()
        
        #first test to see if the action is legal
        new_x, new_y = self.x, self.y
        
        #regardless of legality of move, animate the sprite
        
        #move right
        if action=='d':
            pyg.changeSpriteImage(self.sprite, 0*8+frame)    
            new_x += self.speed
            
        #move down
        elif action=='s':
            pyg.changeSpriteImage(self.sprite, 1*8+frame)    
            new_y += self.speed
            
        #move left
        elif action=='a':
            pyg.changeSpriteImage(self.sprite, 2*8+frame)    
            new_x -= self.speed
            
        #move up
        elif action=='w':
            pyg.changeSpriteImage(self.sprite, 3*8+frame)
            new_y -= self.speed
            
        #stay still
        else:
            pyg.changeSpriteImage(self.sprite, 1 * 8 + 5)
            
        #only allow movement if not incapacitated
        if not self.is_incapacitated:
            #determine which tile/grid of the map would this put the player in    
            tile_col, tile_row = self.the_map.xy_to_cr(new_x, new_y)

            speed = self.the_map.get_speed(tile_col, tile_row)

            self.in_enemy_territory = self.the_map.in_enemy_territory(self.team, tile_col)
            self.in_flag_area = self.the_map.in_flag_area(self.team, tile_col, tile_row)

            if speed:
                #for debugging
                if self.config.verbose and speed != self.speed:
                    print('%s player %d moved from %s to %s' % (self.team, self.player_idx, self.speed_terr[self.speed], self.speed_terr[speed]))

                self.speed = speed
                self.x = new_x
                self.y = new_y

                pyg.moveSprite(self.sprite, self.x, self.y)
            
        
    def get_action(self):
        pass
    
    
    def update_sprite(self, sprite):
        pyg.hideSprite(self.sprite)
        self.sprite = sprite
        pyg.moveSprite(self.sprite, self.x, self.y, centre=True)
        pyg.showSprite(self.sprite)
        
    
    
class HumanPlayer(Player):
    def __init__(self, x, y, idx, team, the_map, config):
        super().__init__(x, y, idx, team, the_map, config)
        #load sprites
        #there are flag holding, nonflag holding, and incapacitated sprites for blue player (human), blue agent, red agent
        #current sprite may change to flag holding or incapacitated sprite
        self.default_sprite = pyg.makeSprite(self.config.blue_player_sprite_path, 32)
        self.holding_flag_sprite = pyg.makeSprite(self.config.blue_player_with_flag_sprite_path, 32)
        self.incapacitated_sprite = pyg.makeSprite(self.config.blue_player_incapacitated_sprite_path, 32)
                                                   
        self.sprite = self.default_sprite
        
        pyg.moveSprite(self.sprite, x, y, centre=True)
        pyg.showSprite(self.sprite)
        
        
    def get_action(self):
        new_x, new_y = self.x, self.y
        
        #move right
        if pyg.keyPressed("d"):
            return 'd'
            
        #move down
        elif pyg.keyPressed("s"):
            return 's'
            
        #move left
        elif pyg.keyPressed("a"):
            return 'a'
            
        #move up
        elif pyg.keyPressed("w"):
            return 'w'
    
    
    
class AgentPlayer(Player):
    def __init__(self, x, y, idx, team, the_map, config):
        super().__init__(x, y, idx, team, the_map, config)
        self.prev_dir = 'a' if team=='red' else 'd'
        
        #load sprites
        #there are flag holding, nonflag holding, and incapacitated sprites for blue player (human), blue agent, red agent
        #current sprite may change to flag holding or incapacitated sprite
        if team=='blue':
            self.default_sprite = pyg.makeSprite(self.config.blue_agent_sprite_path, 32)
            self.holding_flag_sprite = pyg.makeSprite(self.config.blue_agent_with_flag_sprite_path, 32)
            self.incapacitated_sprite = pyg.makeSprite(self.config.blue_agent_incapacitated_sprite_path, 32)
        else:
            self.default_sprite = pyg.makeSprite(self.config.red_agent_sprite_path, 32)
            self.holding_flag_sprite = pyg.makeSprite(self.config.red_agent_with_flag_sprite_path, 32)
            self.incapacitated_sprite = pyg.makeSprite(self.config.red_agent_incapacitated_sprite_path, 32)
            
        self.sprite = self.default_sprite
        
        pyg.moveSprite(self.sprite, x, y, centre=True)
        pyg.showSprite(self.sprite)
        
        
    def get_action(self):
        if random.randint(1,20) == 1:
            self.prev_dir = random.choice(['a','w','s','d'])
            
        return self.prev_dir

In [4]:
#MAKE SURE any changes here are reflected in app/capture_the_flag.py

class CaptureTheFlag():
    def __init__(self, config, new_map=False):
        '''
        Load background, icon, music, sounds, create flag sprites and areas, create players
        '''
        self.config = config
        
        #sounds
        self.grabbed_flag_sound = pyg.makeSound(self.config.grabbed_flag_sound)
        self.incapacitated_sound = pyg.makeSound(self.config.incapacitated_sound)
        self.dropped_flag_sound = pyg.makeSound(self.config.dropped_flag_sound)
        self.tagged_sound = pyg.makeSound(self.config.tagged_sound)
        self.tagged_flag_carrier_sound = pyg.makeSound(self.config.tagged_flag_carrier_sound)
        self.revived_sound = pyg.makeSound(self.config.revived_sound)
        self.victory_sound = pyg.makeSound(self.config.victory_sound)
        
        #for drawing the divide and knowing when someone is off sides
        self.screen_divide_x = (self.config.screen_width//2)
        
        #for debug
        self.speed_terr = {v:k for k,v in self.config.terrain_speeds.items()}
        
        #stores location of terrain speeds including 0 (not allowed areas), flag location, player locations
        self.the_map = TheMap(self.config, new_map)
        
        
        #set screen, title, icon, background, music
        self.__setup(self.the_map.map_path)

        
        #make flags
        self.blue_flag_sprite, self.red_flag_sprite = self.__make_flags()
        
        #update the map object with their locations
        self.the_map.set_flag_location('blue', self.blue_flag_x, self.blue_flag_y)
        self.the_map.set_flag_location('red', self.red_flag_x, self.red_flag_y)
        
        
        #make players with reference to map, and with their locations added to the map
        allowed_blue_sprite_init_tiles = self.__get_allowed_sprite_init_tiles('blue')
    
        #create user player on blue team
        self.user_player = self.__make_player(allowed_blue_sprite_init_tiles, idx=0, team='blue', agent=False)
        #set location in map
        self.the_map.player_tile = (self.user_player.tile_row, self.user_player.tile_col)
        
        #create other blue players
        self.blue_players = [self.user_player]
        for i in range(2):#self.config.blue_team_size - 1):
            blue_player = self.__make_player(allowed_blue_sprite_init_tiles, idx=i+1, team='blue')
            self.blue_players.append(blue_player)
            #add to map
            self.the_map.blue_agent_tiles.append((blue_player.tile_row, blue_player.tile_col))

            
        #get red side non-lake non border tile locations
        allowed_red_sprite_init_tiles = self.__get_allowed_sprite_init_tiles('red')
        
        #create red players
        self.red_players = []
        for i in range(3):#self.config.red_team_size):
            red_player = self.__make_player(allowed_red_sprite_init_tiles, idx=i, team='red')
            self.red_players.append(red_player)
            self.the_map.red_agent_tiles.append((red_player.tile_row, red_player.tile_col))
            
        #all players
        self.players = self.blue_players + self.red_players
        
        #a double check to ensure multiple players can't have same flag
        self.blue_flag_in_play = False
        self.red_flag_in_play = False
        
    
    def run(self):
        nextFrame = pyg.clock()
        frame = 0
        
        while True:
            #quit?
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                    
            # We only animate our character every 80ms.
            if pyg.clock() > nextFrame:
                self.__draw_divide()
                self.__draw_flag_areas()
                # There are 8 frames of animation in each direction so the modulus 8 allows it to loop                        
                frame = (frame+1)%8                         
                nextFrame += 80 

            #execute each players action
            for player in self.players:
                #print('updating player %d' % player.player_idx)
                player.update(frame)
            
            #game logic here - IN PROGRESS
            state = self.__get_state()
            
            #handle states
            if state['blue_wins']:
                pygame.mixer.music.fadeout(1000)
                pyg.pause(1000)
                win_label = pyg.makeLabel('BLUE WINS!!!', 80, self.config.screen_width//2-200, self.config.screen_height//2-40, 
                              fontColour='blue', font='Arial', background="black")
                pyg.showLabel(win_label)
                self.end_game_music = pyg.makeMusic(self.config.end_game_music)
                pyg.playMusic()
                pyg.pause(5000)
                pygame.mixer.music.fadeout(3000)
                pyg.pause(3000)
                break
            elif state['red_wins']:
                pygame.mixer.music.fadeout(1000)
                pyg.pause(1000)
                win_label = pyg.makeLabel('RED WINS!!!', 80, self.config.screen_width//2-200, self.config.screen_height//2-40, 
                              fontColour='red', font='Arial', background="black")
                pyg.showLabel(win_label)
                self.end_game_music = pyg.makeMusic(self.config.end_game_music)
                pyg.playMusic()
                pyg.pause(5000)
                pygame.mixer.music.fadeout(3000)
                pyg.pause(3000)
                break
           

            pyg.tick(10)

        pyg.endWait()
        
        
    def __get_state(self):
        '''Game logic here.'''
        state = {'blue_wins':False, 'red_wins':False}
        
        #all states can be determined by what the players are touching and where they are
        for player in self.players:
            
            #can player do anything?
            if player.is_incapacitated:
                player.incapacitated_countdown -= 1
                
                #is countdown over?
                if player.incapacitated_countdown<=0:
                    print('%s player is no longer incapacitated' % player.team)
                    self.__handle_revival(player)
                #player revived - TODO use sprite group
                elif self.__tagged_by_team_member(player):
                    print('%s player revived' % player.team)
                    self.__handle_revival(player)
                continue
                
                
            #check for win condition
            if player.has_flag and player.in_flag_area:
                if player.team=='blue':
                    print('blue wins')
                    state['blue_wins']=True
                else:
                    print('red wins')
                    state['red_wins']=True
                break
                
            
            #this player isn't touching anything
            if not pyg.allTouching(player.sprite):
                continue
                
                
            #flag grabbed?
            if not self.blue_flag_in_play and player.team=='red' and pyg.touching(player.sprite, self.blue_flag_sprite):
                print('%s player %d touched blue flag' % (player.team, player.player_idx))
                self.__handle_flag_grabbed(player, self.blue_flag_sprite)
                continue
            elif not self.red_flag_in_play and player.team=='blue' and pyg.touching(player.sprite, self.red_flag_sprite):
                print('%s player %d touched blue flag' % (player.team, player.player_idx))
                self.__handle_flag_grabbed(player, self.red_flag_sprite)
                continue
        
        
            #check for player tagged - TODO make team player sprite Groups instead of looping
            if player.in_enemy_territory or player.has_flag:
                if player.team=='blue':
                    if any([pyg.touching(player.sprite, red_player.sprite)!=None for red_player in self.red_players]):
                        print('blue player tagged')
                        self.__handle_tag(player)
                        continue
                elif player.team=='red':
                    if any([pyg.touching(player.sprite, blue_player.sprite)!=None for blue_player in self.blue_players]):
                        print('red player tagged')
                        self.__handle_tag(player)
                        continue
        
        return state
    
    
    #state actions
    
    def __handle_flag_grabbed(self, player, flag_sprite):
        pyg.playSound(self.grabbed_flag_sound)
        pyg.hideSprite(flag_sprite)
        player.has_flag = True
        #player.sprite = player.holding_flag_sprite
        player.update_sprite(player.holding_flag_sprite)
        if player.team=='blue':
            self.red_flag_in_play = False
        else:
            self.blue_flag_in_play = False
        
        
    def __handle_tag(self, player):
        if player.has_flag:
            pyg.playSound(self.tagged_flag_carrier_sound)
            player.has_flag = False
            
            #return flag (or drop it near them? then it will never go backwards)
            if player.team=='blue':
                pyg.moveSprite(self.red_flag_sprite, x=self.red_flag_x, y=self.red_flag_y, centre=True)
                pyg.showSprite(self.red_flag_sprite)
                self.red_flag_in_play = False
            else:
                pyg.moveSprite(self.blue_flag_sprite, x=self.blue_flag_x, y=self.blue_flag_y, centre=True)
                pyg.showSprite(self.blue_flag_sprite)
                self.blue_flag_in_play = False
        else:
            pyg.playSound(self.tagged_sound)
            
        pyg.playSound(self.incapacitated_sound)
        
        player.is_incapacitated = True
        #player.energy = 0
        player.update_sprite(player.incapacitated_sprite)
        player.incapacitated_countdown = 60
        
            
    def __tagged_by_team_member(self, player):
        if player.team=='blue':
            for blue_player in self.blue_players:
                #can't revive yourself!!!
                if not player==blue_player and pyg.touching(player.sprite, blue_player.sprite):
                    return True
        else:
            for red_player in self.red_players:
                if not player==red_player and pyg.touching(player.sprite, red_player.sprite):
                    return True
                
        return False
                                                            
            
    def __handle_revival(self, player):
        pyg.playSound(self.revived_sound)
        player.is_incapacitated = False
        #player.energy = self.config.player_max_energy
        player.update_sprite(player.default_sprite)
        player.incapacitated_countdown = 0
        
        
    def __setup(self, map_path):
        pyg.screenSize(self.config.screen_width, self.config.screen_height)
        pyg.setIcon(self.config.game_icon_image_path)
        pyg.setWindowTitle(self.config.game_name)
        
        #load the map
        pyg.setBackgroundImage(map_path)
        
        #draw dividing line
        self.__draw_divide()
        
        #music
        pyg.makeMusic(self.config.default_game_music)
        pyg.playMusic(loops=-1)
        #TODO
        #self.single_flag_runner_music = pyg.makeMusic(self.config)
        #self.double_flag_runner_music = pyg.makeMusic(self.config)
        
        
    def __draw_divide(self):
        pyg.drawLine(self.screen_divide_x-3, self.config.map_border_size,
                     self.screen_divide_x-3, self.config.screen_height - self.config.map_border_size, 
                     'blue', linewidth=3)
        
        pyg.drawLine(self.screen_divide_x, self.config.map_border_size, 
                     self.screen_divide_x, self.config.screen_height - self.config.map_border_size, 
                     'red', linewidth=3)
    
    
    def __make_flags(self):
        self.blue_flag_x, self.blue_flag_y = self.__get_flag_position(
            self.the_map, 'blue')
        self.red_flag_x, self.red_flag_y = self.__get_flag_position(
            self.the_map, 'red')
        
        self.__draw_flag_areas()
        
        blue_flag_sprite = pyg.makeSprite(self.config.blue_flag_sprite_path)
        pyg.moveSprite(blue_flag_sprite, x=self.blue_flag_x, y=self.blue_flag_y, centre=True)
        pyg.showSprite(blue_flag_sprite)
        
        red_flag_sprite = pyg.makeSprite(self.config.red_flag_sprite_path)
        pyg.moveSprite(red_flag_sprite, x=self.red_flag_x, y=self.red_flag_y, centre=True)
        pyg.showSprite(red_flag_sprite)
        
        return blue_flag_sprite, red_flag_sprite
        
        
    def __get_flag_position(self, the_map, team):
        #choose an available row along a column a set distance from the border
        pad = 20
        
        if team=='blue':
            x = the_map.border_size + self.config.flag_area_size//2 + pad
        else:
            x = self.config.screen_width - the_map.border_size - pad - self.config.flag_area_size//2
            
        flag_tile_c = x//the_map.tile_size
        
        col_speeds = the_map.tile_speeds[:, flag_tile_c]
        idx = np.where(col_speeds > 0)[0].tolist()
        
        #trim top and bottow 2 options to avoid map area going off the screen
        flag_tile_r = random.choice(idx[2:-2])
        y = flag_tile_r * the_map.tile_size
        
        return x, y

    
    def __draw_flag_areas(self):
        pyg.drawEllipse(self.blue_flag_x, self.blue_flag_y, self.config.flag_area_size, self.config.flag_area_size, 
                        colour='blue', linewidth=3)
        
        pyg.drawEllipse(self.red_flag_x, self.red_flag_y, self.config.flag_area_size, self.config.flag_area_size, 
                        colour='red', linewidth=3)
        
        
    def __get_allowed_sprite_init_tiles(self, team):
        side = self.the_map.tile_speeds.shape[1]//3
        if team=='blue':
            idx = np.where(self.the_map.tile_speeds[:,:side]>0)
        else:
            idx = np.where(self.the_map.tile_speeds[:,(side*2):]>0)
            idx = (idx[0], idx[1] + np.ones_like(idx[1]) * (side*2))
            
        allowed_init_tiles = list(zip(idx[0].tolist(), idx[1].tolist()))
        
        random.shuffle(allowed_init_tiles)
        
        return allowed_init_tiles


    def __make_player(self, allowed_sprite_init_tiles, idx=-1, team='', agent=True):
        player_r_idx, player_c_idx = allowed_sprite_init_tiles.pop()

        player_x_pos = (player_c_idx * self.config.terrain_tile_size) + self.config.terrain_tile_size//2
        player_y_pos = (player_r_idx * self.config.terrain_tile_size) + self.config.terrain_tile_size//2
        print('%s %s on r=%s, c=%d, x=%d, y=%d' % (team, 'agent' if agent else 'player', 
                                                   player_r_idx, player_c_idx, player_x_pos, player_y_pos))

        if not agent: #this will be a different sprite
            player = HumanPlayer(player_x_pos, player_y_pos, idx, team, self.the_map, self.config)
        else:
            player = AgentPlayer(player_x_pos, player_y_pos, idx, team, self.the_map, self.config)

        return player

In [4]:
%tb

config = Config(verbose=True)
game = CaptureTheFlag(config, new_map=False)
game.run()

No traceback available to show.


Using default map
blue flag is on 14, 4, in area [(13, 3), (13, 4), (13, 5), (14, 3), (14, 4), (14, 5), (15, 3), (15, 4), (15, 5)]
red flag is on 19, 37, in area [(18, 36), (18, 37), (18, 38), (19, 36), (19, 37), (19, 38), (20, 36), (20, 37), (20, 38)]
blue player on r=7, c=9, x=190, y=150
blue agent on r=6, c=8, x=170, y=130
blue agent on r=22, c=12, x=250, y=450
blue agent on r=28, c=8, x=170, y=570
blue agent on r=28, c=3, x=70, y=570
red agent on r=12, c=35, x=710, y=250
red agent on r=29, c=37, x=750, y=590
red agent on r=16, c=40, x=810, y=330
red agent on r=8, c=37, x=750, y=170
red agent on r=1, c=31, x=630, y=30
blue player 2 moved from plain to hill
blue player 3 moved from plain to hill
red player 1 moved from plain to swamp
red player 0 moved from plain to hill
red player 4 moved from plain to hill
blue player 1 moved from plain to swamp
red player 3 moved from plain to swamp
red player 2 moved from plain to hill
blue player 4 moved from plain to hill
red player 1 moved fro

In [5]:
%tb

pygame.init()
pygame.mixer.init()

config = Config(verbose=True)
game = CaptureTheFlag(config, new_map=False)
game.run()

pygame.quit()
game=None

No traceback available to show.


Using default map
blue flag is on 23, 4, in area [(22, 3), (22, 4), (22, 5), (23, 3), (23, 4), (23, 5), (24, 3), (24, 4), (24, 5)]
red flag is on 27, 37, in area [(26, 36), (26, 37), (26, 38), (27, 36), (27, 37), (27, 38), (28, 36), (28, 37), (28, 38)]
blue player on r=12, c=10, x=210, y=250
blue agent on r=10, c=10, x=210, y=210
blue agent on r=13, c=13, x=270, y=270
red agent on r=18, c=34, x=690, y=370
red agent on r=1, c=39, x=790, y=30
red agent on r=3, c=38, x=770, y=70
blue player 2 moved from plain to mountain
red player 0 moved from plain to swamp
red player 2 moved from plain to swamp
red player 1 moved from plain to swamp
blue player 1 moved from plain to hill
red player 0 moved from swamp to plain
red player 0 moved from plain to swamp
blue player 2 moved from mountain to hill
red player 2 moved from swamp to plain
blue player 1 moved from hill to mountain
red player 2 moved from plain to hill
blue player 2 moved from hill to plain
blue player 2 moved from plain to hill
blu